# Introduction to DataFrames
**[Bogumił Kamiński](http://bogumilkaminski.pl/about/), Apr 21, 2018**

In [1]:
using DataFrames # load package

## Reshaping DataFrames

### Wide to long

In [2]:
x = DataFrame(id=[1,2,3,4], id2=[1,1,2,2], M1=[11,12,13,14], M2=[111,112,113,114])

,id,id2,M1,M2
1,1,1,11,111
2,2,1,12,112
3,3,2,13,113
4,4,2,14,114


In [3]:
melt(x, :id, [:M1, :M2]) # first pass id-variables and then measure variables; meltdf makes a view

,variable,value,id
1,M1,11,1
2,M1,12,2
3,M1,13,3
4,M1,14,4
5,M2,111,1
6,M2,112,2
7,M2,113,3
8,M2,114,4


In [4]:
# optionally you can rename columns; melt and stack are identical but order of arguments is reversed
stack(x, [:M1, :M2], :id, variable_name=:key, value_name=:observed) # first measures and then id-s; stackdf creates view

,key,observed,id
1,M1,11,1
2,M1,12,2
3,M1,13,3
4,M1,14,4
5,M2,111,1
6,M2,112,2
7,M2,113,3
8,M2,114,4


In [5]:
# if second argument is omitted in melt or stack , all other columns are assumed to be the second argument
# but measure variables are selected only if they are <: AbstractFloat
melt(x, [:id, :id2])

,variable,value,id,id2
1,M1,11,1,1
2,M1,12,2,1
3,M1,13,3,2
4,M1,14,4,2
5,M2,111,1,1
6,M2,112,2,1
7,M2,113,3,2
8,M2,114,4,2


In [6]:
melt(x, [1, 2]) # you can use index instead of symbol

,variable,value,id,id2
1,M1,11,1,1
2,M1,12,2,1
3,M1,13,3,2
4,M1,14,4,2
5,M2,111,1,1
6,M2,112,2,1
7,M2,113,3,2
8,M2,114,4,2


In [7]:
bigx = DataFrame(rand(10^6, 10)) # a test comparing creation of new DataFrame and a view
bigx[:id] = 1:10^6
@time melt(bigx, :id)
@time melt(bigx, :id)
@time meltdf(bigx, :id)
@time meltdf(bigx, :id);

  0.288607 seconds (25.97 k allocations: 230.395 MiB, 31.10% gc time)
  0.252100 seconds (127 allocations: 228.889 MiB, 51.72% gc time)
  0.258912 seconds (102.99 k allocations: 5.480 MiB)
  0.000139 seconds (85 allocations: 5.250 KiB)


In [8]:
x = DataFrame(id = [1,1,1], id2=['a','b','c'], a1 = rand(3), a2 = rand(3))

,id,id2,a1,a2
1,1,'a',0.458145,0.142355
2,1,'b',0.799502,0.631973
3,1,'c',0.665952,0.204755


In [9]:
melt(x)

,variable,value,id,id2
1,a1,0.458145,1,'a'
2,a1,0.799502,1,'b'
3,a1,0.665952,1,'c'
4,a2,0.142355,1,'a'
5,a2,0.631973,1,'b'
6,a2,0.204755,1,'c'


In [10]:
melt(DataFrame(rand(3,2))) # by default stack and melt treats floats as value columns

,variable,value
1,x1,0.284223
2,x1,0.498334
3,x1,0.84188
4,x2,0.845803
5,x2,0.722629
6,x2,0.995074


In [11]:
df = DataFrame(rand(3,2))
df[:key] = [1,1,1]
mdf = melt(df) # duplicates in key are silently accepted

,variable,value,key
1,x1,0.876846,1
2,x1,0.0686333,1
3,x1,0.0815149,1
4,x2,0.984013,1
5,x2,0.139363,1
6,x2,0.859373,1


### Long to wide

In [12]:
x = DataFrame(id = [1,1,1], id2=['a','b','c'], a1 = rand(3), a2 = rand(3))

,id,id2,a1,a2
1,1,'a',0.754369,0.095943
2,1,'b',0.494067,0.227082
3,1,'c',0.622234,0.791082


In [13]:
y = melt(x, [1,2])
display(x)
display(y)

,id,id2,a1,a2
1,1,'a',0.754369,0.095943
2,1,'b',0.494067,0.227082
3,1,'c',0.622234,0.791082


,variable,value,id,id2
1,a1,0.754369,1,'a'
2,a1,0.494067,1,'b'
3,a1,0.622234,1,'c'
4,a2,0.095943,1,'a'
5,a2,0.227082,1,'b'
6,a2,0.791082,1,'c'


In [14]:
unstack(y, :id2, :variable, :value) # stndard unstack with a unique key

,id2,a1,a2
1,'a',0.754369,0.095943
2,'b',0.494067,0.227082
3,'c',0.622234,0.791082


In [15]:
unstack(y, :variable, :value) # all other columns are treated as keys

,id,id2,a1,a2
1,1,'a',0.754369,0.095943
2,1,'b',0.494067,0.227082
3,1,'c',0.622234,0.791082


In [16]:
# by default :id, :variable and :value names are assumed; in this case it produces duplicate keys
unstack(y)

,id,a1,a2
1,1,0.622234,0.791082


In [17]:
df = stack(DataFrame(rand(3,2)))

,variable,value
1,x1,0.678752
2,x1,0.929795
3,x1,0.884891
4,x2,0.925154
5,x2,0.474071
6,x2,0.731631


In [18]:
unstack(df, :variable, :value) # unable to unstack when no key column is present

LoadError: [91mArgumentError: No key column found[39m